In [1]:
# Install once only - python official library from AWS
# !pip install boto3

### define configuration

In [2]:
aws_access_key_id = <Your Access Key>
aws_secret_access_key = <Your secret access key>
myregion= <your region>

In [3]:
import boto3

In [17]:
from botocore.exceptions import ClientError, WaiterError

In [20]:
import logging

In [21]:
logger = logging.getLogger(__name__)

In [ ]:
import os
from datetime import datetime
def setup_logger(log_directory='C:/MyPythonProjects/AWS_SES_Logging'):  # Example path - change this to your preferred path
    """
    Set up logger with proper formatting and handlers
    Args:
        log_directory (str): Directory where log files will be stored
    """
    # Create logs directory if it doesn't exist
    if not os.path.exists(log_directory):
        os.makedirs(log_directory)
    
    # Create a timestamp for the log file name
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_file_path = os.path.join(log_directory, f'ses_email_sender_{timestamp}.log')
    
    # Configure logger
    logger = logging.getLogger('ses_email_sender')
    logger.setLevel(logging.INFO)
    
    # Create console handler with formatting
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    
    # Create file handler
    file_handler = logging.FileHandler(log_file_path)
    file_handler.setLevel(logging.INFO)
    
    # Create formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(formatter)
    file_handler.setFormatter(formatter)
    
    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(file_handler)
    
    # Log the locations where logs are being written
    logger.info(f"Logging to console and file: {log_file_path}")
    
    return logger

In [ ]:
logger = setup_logger(log_directory='C:/MyLogs/SESEmailLogs')#<Put your own preferred log directory path>

In [ ]:
def Send_SES_Email(aws_access, aws_secret, aws_region, vsource, vTo, vCC, vBCC, vsubject, vtext, vhtml):
    """
    Send email using AWS SES with comprehensive logging
    
    Args:
        aws_access (str): AWS access key ID
        aws_secret (str): AWS secret access key
        aws_region (str): AWS region name
        vsource (str): Sender email address
        vTo (list): List of recipient email addresses
        vCC (list): List of CC email addresses
        vBCC (list): List of BCC email addresses
        vsubject (str): Email subject
        vtext (str): Plain text email body
        vhtml (str): HTML email body
    
    Returns:
        str: Message ID if successful
        
    Raises:
        ClientError: If there's an error with AWS SES
        Exception: For other unexpected errors
    """
    logger = logging.getLogger('ses_email_sender')
    
    # Log initial attempt
    logger.info(f"Attempting to send email from {vsource} to {vTo}")
    logger.debug(f"CC: {vCC}, BCC: {vBCC}")
    
    try:
        # Create SES client
        logger.debug(f"Creating SES client for region {aws_region}")
        ses_client = boto3.client(
            'ses',
            aws_access_key_id=aws_access,
            aws_secret_access_key=aws_secret,
            region_name=aws_region
        )
        
        # Prepare email
        logger.debug("Preparing email payload")
        response = ses_client.send_email(
            Source=vsource,
            Destination={
                'ToAddresses': vTo,
                'CcAddresses': vCC,
                'BccAddresses': vBCC
            },
            Message={
                "Subject": {"Data": vsubject},
                "Body": {
                    "Text": {"Data": vtext},
                    "Html": {"Data": vhtml}
                },
            }
        )
        
        message_id = response["MessageId"]
        logger.info(f"Successfully sent email with MessageId: {message_id}")
        logger.debug(f"Full response: {response}")
        
    except ClientError as e:
        logger.error(f"Failed to send email from {vsource}")
        logger.error(f"Error: {str(e)}")
        logger.debug(f"Error details: {e.response['Error']}")
        raise
    except Exception as e:
        logger.error(f"Unexpected error while sending email: {str(e)}")
        raise
    else:
        return message_id

In [46]:
Send_SES_Email(aws_access_key_id,aws_secret_access_key,'ap-southeast-1','test@test.com',['xxxx@gmail.com','test@xyz.com'],[],[],'Test Email Subject 5','This is a test email body 5','This is a test email html body 5')

'010e018e039375e6-a2a37041-f498-4938-af11-9f0168b781ab-000000'

In [ ]:
message_id = Send_SES_Email(
    aws_access="your_aws_access_key",
    aws_secret="your_aws_secret_key",
    aws_region="your_aws_region",
    vsource="sender@example.com",
    vTo=["recipient1@example.com", "recipient2@example.com"],
    vCC=["cc1@example.com"],
    vBCC=["bcc1@example.com"],
    vsubject="Hello from Python and AWS SES",
    vtext="This is a plain text email sent via Python and AWS SES.",
    vhtml="<p>This is an HTML email sent via Python and AWS SES.</p>")
print("Email sent successfully! Message ID:", message_id)